In [1]:
import librosa
import numpy as np
import soundfile as sf
import os
import noisereduce as nr  

def enhance_audio(input_path, output_path):
    try:
        output_dir = os.path.dirname(output_path)
        os.makedirs(output_dir, exist_ok=True)
        noisy_audio, sr = librosa.load(input_path, sr=None)
        reduced_noise = nr.reduce_noise(y=noisy_audio, sr=sr)
        sf.write(output_path, reduced_noise, sr)
        print(f"Enhanced audio saved to {output_path}")       
    except Exception as e:
        print(f"Error processing {input_path}: {str(e)}")
        
input_directory = 'C:/Users/ABHISHEK/program_1/trained_trimmed_data'
output_directory = 'C:/Users/ABHISHEK/program_1/trained_trimmed_data_enhanced'
audio_files = [f for f in os.listdir(input_directory) if f.endswith('.wav')]

for audio_file in audio_files:
    input_path = os.path.join(input_directory, audio_file)
    output_path = os.path.join(output_directory, os.path.splitext(audio_file)[0] + '_enhanced.wav')
enhance_audio(input_path, output_path)

Enhanced audio saved to C:/Users/ABHISHEK/program_1/trained_trimmed_data_enhanced\336_depressed-[AudioTrimmer.com]_enhanced.wav


In [6]:
import os
import numpy as np
from python_speech_features import mfcc
import soundfile as sf
import librosa
from python_speech_features import logfbank

input_directory = 'C:/Users/ABHISHEK/program_1/tested_trimmed_data_enhanced'
output_directory = 'C:/Users/ABHISHEK/program_1/tested_trimmed_data_enhanced_mfcccoff'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

audio_files = [f for f in os.listdir(input_directory) if f.endswith('.wav')]
num_ceps = 12  # Number of cepstral coefficients to keep

for audio_file in audio_files:
    audio, sr = librosa.load(os.path.join(input_directory, audio_file), sr=None)
    log_filter_bank = logfbank(audio, samplerate=sr, nfilt=num_ceps)
    log_filter_bank = np.power(log_filter_bank, 1.5)
    mfcc_features = mfcc(audio, samplerate=sr, numcep=num_ceps)
output_file = os.path.join(output_directory, os.path.splitext(audio_file)[0] + '_mfcc.npy')
np.save(output_file, mfcc_features)

C:\Users\ABHISHEK\AppData\Local\Temp\ipykernel_2524\812771676.py:30: RuntimeWarning: invalid value encountered in power
  log_filter_bank = np.power(log_filter_bank, 1.5)


In [5]:
import os
from pydub import AudioSegment
import numpy as np

def calculate_zcr(audio_data):
    # Calculate ZCR by counting the number of zero-crossings
    zcr_count = np.sum(np.diff(np.sign(audio_data)) != 0)
    # Normalize by the length of the signal
    zcr = zcr_count / len(audio_data)
    return zcr

input_directory = 'C:/Users/ABHISHEK/program_1/trained_trimmed_data_enhanced'
output_directory = 'C:/Users/ABHISHEK/program_1/trained_trimmed_data_enhanced_ZCR'

os.makedirs(output_directory, exist_ok=True)
audio_files = [f for f in os.listdir(input_directory) if f.endswith('.wav')]

for audio_file in audio_files:
    audio = AudioSegment.from_wav(os.path.join(input_directory, audio_file))
    audio_data = np.array(audio.get_array_of_samples())
    zcr_value = calculate_zcr(audio_data)

    output_filename = os.path.splitext(audio_file)[0] + '_zcr.csv'
    output_path = os.path.join(output_directory, output_filename)
    
    np.savetxt(output_path, [zcr_value], delimiter=',', fmt='%.2f')
    print(f"Processed {audio_file} and saved ZCR value to {output_path}")
    
print("Processing complete.")

Processed 303 NOT Depressed-[AudioTrimmer.com]_enhanced.wav and saved ZCR value to C:/Users/ABHISHEK/program_1/trained_trimmed_data_enhanced_ZCR\303 NOT Depressed-[AudioTrimmer.com]_enhanced_zcr.csv
Processed 304 depressed-[AudioTrimmer.com]_enhanced.wav and saved ZCR value to C:/Users/ABHISHEK/program_1/trained_trimmed_data_enhanced_ZCR\304 depressed-[AudioTrimmer.com]_enhanced_zcr.csv
Processed 305_Depressed-[AudioTrimmer.com]_enhanced.wav and saved ZCR value to C:/Users/ABHISHEK/program_1/trained_trimmed_data_enhanced_ZCR\305_Depressed-[AudioTrimmer.com]_enhanced_zcr.csv
Processed 310_Depressed-[AudioTrimmer.com]_enhanced.wav and saved ZCR value to C:/Users/ABHISHEK/program_1/trained_trimmed_data_enhanced_ZCR\310_Depressed-[AudioTrimmer.com]_enhanced_zcr.csv
Processed 312_NotDepressed-[AudioTrimmer.com]_enhanced.wav and saved ZCR value to C:/Users/ABHISHEK/program_1/trained_trimmed_data_enhanced_ZCR\312_NotDepressed-[AudioTrimmer.com]_enhanced_zcr.csv
Processed 313_depressed-[AudioT

In [ ]:
import os
import librosa
import soundfile as sf
import numpy as np

# Input directory containing audio files
input_directory = 'C:/Users/ABHISHEK/program_1/tested_trimmed_data_enhanced'

# Output directory for saving extracted features
output_directory = 'C:/Users/ABHISHEK/program_1/tested_trimmed_data_enhanced_f0'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Process each audio file in the input directory
audio_files = [f for f in os.listdir(input_directory) if f.endswith('.wav')]

for audio_file in audio_files:
    # Load the audio signal
    audio, sr = librosa.load(os.path.join(input_directory, audio_file), sr=None)

    # Estimate F0 using librosa
    f0, voiced = librosa.piptrack(y=audio, sr=sr)

    # Extract the F0 values from the pitch tracker
    f0_values = []

    for i in range(f0.shape[1]):
        f0_frame = f0[:, i][voiced[:, i] > 0]
        if len(f0_frame) > 0:
            f0_values.extend(f0_frame)

    # Calculate statistics on the extracted F0 values (e.g., mean)
    if f0_values:
        f0_mean = np.mean(f0_values)
    else:
        f0_mean = 0.0  # Handle cases with no voiced frames

    # Create a new filename for the output
    output_filename = os.path.splitext(audio_file)[0] + '_mean_f0.csv'
    output_path = os.path.join(output_directory, output_filename)

    # Save the mean F0 value to a CSV file
    np.savetxt(output_path, [f0_mean], delimiter=',', fmt='%.2f')

print(f"Processed {audio_file} and saved mean F0 value to {output_path}")
print("Processing complete.")

In [8]:
import os
import numpy as np

mfcc_directory = 'C:/Users/ABHISHEK/program_1/trained_trimmed_data_enhanced_mfcccoff_csv'
fundamental_freq_directory = 'C:/Users/ABHISHEK/program_1/trained_trimmed_data_enhanced_f0'
output_csv_file = 'C:/Users/ABHISHEK/program_1/features_trained_trimmed_train_csv'


mfcc_mean = []
mfcc_variance = []

for csv_file in os.listdir(mfcc_directory):
    if csv_file.endswith('.csv'):
        csv_data = np.loadtxt(os.path.join(mfcc_directory, csv_file), delimiter=',')
        if csv_data.shape[0] > 0:
            all_mfcc_coeffs = csv_data.flatten()
            mfcc_mean.append(np.mean(all_mfcc_coeffs))
            mfcc_variance.append(np.var(all_mfcc_coeffs))

fundamental_freq_mean = []
for csv_file in os.listdir(fundamental_freq_directory):
    if csv_file.endswith('.csv'):
        csv_data = np.loadtxt(os.path.join(fundamental_freq_directory, csv_file), delimiter=',')
        mean_value = np.mean(csv_data)  # Calculate mean fundamental frequency
        fundamental_freq_mean.append(mean_value)
combined_features = np.column_stack((mfcc_mean, mfcc_variance, fundamental_freq_mean))
np.savetxt(output_csv_file, combined_features, delimiter=',', fmt='%.6f', comments='')
print("Combined features saved to", output_csv_file)

Combined features saved to C:/Users/ABHISHEK/program_1/features_trained_trimmed_train_csv
